In [137]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import re
from jinja2 import Environment, FileSystemLoader



%matplotlib inline
%load_ext blackcellmagic

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The blackcellmagic extension is already loaded. To reload it, use:
  %reload_ext blackcellmagic


In [161]:
wkdir = '/projects/trans_scratch/validations/workspace/szong/Cervical/variant_bwamem/validation/snvs/'
template_dir = '/projects/trans_scratch/validations/workspace/szong/Cervical/variant_bwamem/validation/batch_files/'
template_file = 'batch_file_template_20191015.txt'

# make a list of variants to review

In [186]:
f = '/projects/trans_scratch/validations/workspace/szong/Cervical/variant_bwamem/validation/strelka_HMLM_targets_summary_20191016.txt'
df = pd.read_csv(f, sep='\t', usecols=['chr', 'pos', 'ref', 'alt', 'patient', 'gene'])
df.drop_duplicates(inplace=True)
df.head(2)
df.shape

,chr,pos,ref,alt,patient,gene
0,6,142706206,G,A,HTMCP-03-06-02025,GPR126
5,6,142706206,G,A,HTMCP-03-06-02061,GPR126


(71, 6)

In [187]:
df['start'] = df['pos'] - 150
df['end'] = df['pos'] +150
df = df.astype(str)
df['positions'] = df['chr'] + ':' + df['start'] + '-' + df['end']
df['is_real'] = 'no'
df['comments'] = 'xxx'
df["chromosome"] = df.chr
df.head(2)

,chr,pos,ref,alt,patient,gene,start,end,positions,is_real,comments,chromosome
0,6,142706206,G,A,HTMCP-03-06-02025,GPR126,142706056,142706356,6:142706056-142706356,no,xxx,6
5,6,142706206,G,A,HTMCP-03-06-02061,GPR126,142706056,142706356,6:142706056-142706356,no,xxx,6


In [188]:
patients = df.patient.unique()

In [189]:
for p in patients:
    dff = df.query(f'patient=="{p}"')
    n_muts = dff.shape[0]
    dff.to_csv(f'{wkdir}{p}_{n_muts}.snv', sep='\t',index=False)


# igv batch files

In [165]:
of = '/projects/trans_scratch/validations/workspace/szong/Cervical/variant_bwamem/validation/meta_file_20191016.txt'
df1 = pd.read_csv(of, sep='\t', index_col='Patient')
df1.head(2)

,Tumour,Normal,t_dna,n_dna
Patient,,,,
HTMCP-03-06-02025,B68907,B68872,/projects/analysis/analysis33/B68907/merge4734...,/projects/analysis/analysis33/B68872/merge4731...
HTMCP-03-06-02044,B68919,B68884,/projects/analysis/analysis33/B68919/merge4735...,/projects/analysis/analysis33/B68884/merge4732...


In [179]:
# export
def render_template(template_dir, out_file, template_file,tdna_bam, trna_bam, ndna_bam):
    jinja2_env = Environment(loader=FileSystemLoader([template_dir]),
                             trim_blocks=True)
    template = jinja2_env.get_template(template_file)
    with open(out_file, 'w') as opf:
        content = template.render(out_file=out_file,
                                  tdna_bam=tdna_bam,
                                  trna_bam=trna_bam,
                                  ndna_bam=ndna_bam)
                                                                 
        opf.write(content)
#         print('templated {}'.format(sh_script))
    return out_file

In [183]:
df1.head(2)

,Tumour,Normal,t_dna,n_dna
Patient,,,,
HTMCP-03-06-02025,B68907,B68872,/projects/analysis/analysis33/B68907/merge4734...,/projects/analysis/analysis33/B68872/merge4731...
HTMCP-03-06-02044,B68919,B68884,/projects/analysis/analysis33/B68919/merge4735...,/projects/analysis/analysis33/B68884/merge4732...


In [182]:
for ix, row in df1.iterrows():
    pat = ix
    if pat in patients:
        t_dna = df1.loc[pat, 't_dna']
    #     t_rna = df1.loc[pat, 't_rna']
        n_dna = df1.loc[pat, 'n_dna']
        out_file = f'{template_dir}/{pat}_batch_file.txt'
        _ = render_template(template_dir, out_file, template_file, t_dna, t_rna, n_dna)

# review in igv

the python scripts will read a variants.bed file, which is generated by the scipts, and connect to opened igv session on gpost08. Depending on the review, it will separate the variants into passed and failed files.

currently no good way to go about this. This is primarily due to there is no igv batch command giving me an option to pause for review and then move on to the next position after igv loads all bams and go to the position of interest.

if one patient have many variants, may be just use the existing review scripts:

ssh -X gphost08

igv -b /projects/trans_scratch/validations/workspace/szong/Cervical/mutsig2cv/batch_files/HTMCP-03-06-02063.batch

python /projects/da_workspace/DA-231_SMG_review/variant_igv_review.py -c /projects/tumour_char/analysis_scripts/SVIA/variant_igv_review/tags/v0.2.2/variant_igv_review.cfg -i batch_files/HTMCP-03-06-02063_6.snv -p ./passed_test.txt -f ./failed_test.txt --igv -b ./variants.bed

if reviewing specific variants from many patients, may be just use the following hack around

even this is not a solution

In [35]:
# %%bash
# igv='/gsc/software/linux-x86_64-centos6/jdk1.8.0_102/bin/java -jar /gsc/software/linux-x86_64/IGV-2.3.47/igv.jar'
# $igv -b HTMCP-03-06-02001_batch_file.txt
# read -p "Press enter to continue"
# $igv -b HTMCP-03-06-02002_batch_file.txt
# read -p "Press enter to continue"
# $igv -b HTMCP-03-06-02058_batch_file.txt
